In [20]:
import pandas as pd
import pandas_ta as ta
import glob
from datetime import datetime, time

headers = ['Date','Time','Open','High','Low','Close','Volume']
path = "C:\\Users\\desma\\Downloads\\jupyter work\\DATA\\"
#set up so i could run multiple datasets if wanted
filenames = glob.glob(path + "SPY2h.txt")
for file in filenames:
    print("\nReading file = ", file)
    df = pd.read_csv(file, header = None)
    df.columns=headers
dfLen = len(df)
df


Reading file =  C:\Users\desma\Downloads\jupyter work\DATA\SPY2h.txt


,Date,Time,Open,High,Low,Close,Volume
0,06/22/2018,04:00,256.93,257.43,256.9300,257.2200,100435
1,06/22/2018,06:00,257.22,257.40,257.1200,257.4000,301968
2,06/22/2018,08:00,257.38,257.59,256.8500,257.0300,6437279
3,06/22/2018,10:00,257.02,257.43,256.5600,257.1600,14641972
4,06/22/2018,12:00,257.16,257.46,257.0600,257.3700,5231450
...,...,...,...,...,...,...,...
9139,07/01/2022,10:00,378.69,380.03,373.8000,375.4000,17739835
9140,07/01/2022,12:00,375.44,378.32,374.9001,378.2197,9158204
9141,07/01/2022,14:00,378.23,381.70,377.3000,381.2000,20486865
9142,07/01/2022,16:00,381.20,381.42,380.9000,381.2000,3892847


In [23]:
import numpy as np
tagID = 0
levels = [[0, 1, 0] for i in range(dfLen)]

def close(curr, idx):
    MOE = .0025  #  .3%
    global tagID 
    accumulation = curr
    for row in reversed(range(idx-1)):     #use idx-1, since idx would be current extrema
        if (abs(1-curr/df.iloc[row].Extrema)<MOE and df.iloc[row].Extrema!=0): #find other extrma in bounds
            levels[idx][1] += 1                 #add to count accumulater
            accumulation += df.iloc[row].Extrema     #accumulation of levels within MOE, will later divide by /n
                                            #occurances to compute levels mean
            levels[idx][2] = levels[row][2]   #if found in prior row, level has occured. Copy tagID   
    
    count = levels[idx][1]
    if count>1:        #found other values, and done looping. now can compute the mean
        mean = round(accumulation/count, 2)
        levels[idx][0] = mean
        
    else:     #first occurance of level
        levels[idx][0] = curr
        levels[idx][1] = 1
        levels[idx][2] = tagID
        tagID += 1
    

def support(df, b, curr):
    halfPt = b/2
    test = df.Low[curr-halfPt]
    for i in range(curr-b, curr+1):
        if(test>df.Low[i]): #if we find a value less than our current mark, current != local min
            return 0
    else:
        return 1
        
def resist(df, b, curr):
    halfPt = b/2
    test = df.High[curr-halfPt]
    for i in range(curr-b, curr+1):
        if(test<df.High[i]): #if we find a value greater than our current mark, current != max
            return 0
    else:
        return 1
        
back = 20    #MUSTBE EVEN NUMBER
pos = 10     #CRUCIAL FOR PLACEMENT OF EXTREMA. OTHERWISE BROKEN

S = ['NaN']*dfLen
R = ['NaN']*dfLen
xtrma = [0]*dfLen

for row in range(back, dfLen):
    if(support(df, back, row)==1):
        S[row-pos] = df.Low[row-pos]
        xtrma[row-pos] = df.Low[row-pos] #support level
        
    elif(resist(df, back, row)==1):
        R[row-pos] = df.High[row-pos]
        xtrma[row-pos] = df.High[row-pos]#resistance level
        
df['S'] = S
df['R'] = R
df['Extrema'] = xtrma

#levels = (mean, level tag#, number of instances)
for row in range(0, dfLen):
    if df.iloc[row].Extrema > 0:
        close(df.iloc[row].Extrema, row)
            
df['Level'] = levels
print(levels[dfLen-40:dfLen])
df.tail(40)

C:\Users\desma\AppData\Local\Temp/ipykernel_19136/2077244691.py:11: RuntimeWarning: divide by zero encountered in double_scalars
  if (abs(1-curr/df.iloc[row].Extrema)<MOE and df.iloc[row].Extrema!=0): #find other extrma in bounds


[[0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [393.16, 1, 141], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [372.89, 4, 95], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0]]


,Date,Time,Open,High,Low,Close,Volume,S,R,Extrema,Level
9104,06/27/2022,04:00,392.5700,392.9400,391.0200,391.7000,114682,NaN,NaN,0.00,"[0, 1, 0]"
9105,06/27/2022,06:00,391.7000,391.7600,390.6800,391.4300,259936,NaN,NaN,0.00,"[0, 1, 0]"
9106,06/27/2022,08:00,391.1900,392.7196,387.5200,388.1000,8520688,NaN,NaN,0.00,"[0, 1, 0]"
9107,06/27/2022,10:00,388.1300,391.1300,387.9900,390.5800,13800276,NaN,NaN,0.00,"[0, 1, 0]"
9108,06/27/2022,12:00,390.6000,390.7450,388.1000,388.3918,9454565,NaN,NaN,0.00,"[0, 1, 0]"
9109,06/27/2022,14:00,388.4200,389.3700,387.4400,388.6600,16329407,NaN,NaN,0.00,"[0, 1, 0]"
9110,06/27/2022,16:00,388.6600,389.3500,388.5300,389.1500,3710962,NaN,NaN,0.00,"[0, 1, 0]"
9111,06/27/2022,18:00,389.1500,389.3500,388.7400,389.0900,96640,NaN,NaN,0.00,"[0, 1, 0]"
9112,06/28/2022,04:00,389.6300,391.3600,389.6300,390.2200,68000,NaN,NaN,0.00,"[0, 1, 0]"
9113,06/28/2022,06:00,390.2200,391.1500,390.2200,390.8400,206210,NaN,NaN,0.00,"[0, 1, 0]"


In [24]:
for i in range(dfLen):
    if levels[i][1]>1:
        print(levels[i])

[251.05, 2, 0]
[255.7, 2, 1]
[251.42, 2, 0]
[255.56, 3, 1]
[260.6, 2, 2]
[260.4, 3, 2]
[260.4, 4, 2]
[265.51, 2, 7]
[260.82, 5, 2]
[263.8, 2, 6]
[260.49, 6, 2]
[265.88, 2, 7]
[267.12, 2, 8]
[265.48, 3, 7]
[263.18, 3, 5]
[265.46, 4, 7]
[261.62, 2, 4]
[265.64, 6, 7]
[266.74, 4, 8]
[271.95, 3, 10]
[267.26, 3, 8]
[270.18, 2, 12]
[267.98, 3, 9]
[272.24, 3, 11]
[272.29, 4, 11]
[271.64, 3, 10]
[275.01, 2, 15]
[272.35, 5, 11]
[275.04, 3, 15]
[255.31, 3, 1]
[267.95, 4, 9]
[271.08, 4, 10]
[263.88, 3, 6]
[257.89, 2, 3]
[251.75, 2, 0]
[255.24, 4, 1]
[243.88, 2, 19]
[263.9, 4, 6]
[250.64, 2, 0]
[257.85, 3, 3]
[250.64, 3, 0]
[246.48, 3, 18]
[263.76, 6, 6]
[262.91, 3, 5]
[246.26, 3, 18]
[254.86, 3, 1]
[251.6, 4, 0]
[252.45, 3, 0]
[243.71, 3, 19]
[237.16, 2, 26]
[231.21, 2, 27]
[244.36, 3, 19]
[251.89, 5, 0]
[246.19, 4, 18]
[251.68, 6, 0]
[246.81, 3, 22]
[257.9, 4, 3]
[252.72, 3, 21]
[257.82, 5, 3]
[263.53, 7, 6]
[263.8, 7, 6]
[260.62, 6, 2]
[265.62, 7, 7]
[262.96, 5, 5]
[261.71, 3, 4]
[263.82, 8, 6]


In [ ]:
def SIGNAL():
    return df.Signal

In [ ]:
from backtesting import Strategy, Backtest
import numpy as np

class myStrat(Strategy):
    def init(self):
        super().init()
        self.signal1=self.I(SIGNAL)     
        
    def next(self):
        super().next()
        #slatr = 2*self.data.ATR[-1]
        #slpct = self.data.Close[-1]*.02
        #tpslATR = 1.5
        trailSL = self.data.Close[-1]*.005
        for trade in self.trades:
            if trade.is_long:
                trade.sl = max(trade.sl or -np.inf, self.data.Close[-1] - trailSL)
            else:
                trade.sl = min(trade.sl or np.inf, self.data.Close[-1] + trailSL)
        
        if self.signal1==1 and len(self.trades)==0: #Bullish Signal
            self.buy()#, tp=tp1)
            
        elif self.signal1==2 and len(self.trades)==0: #Bearish Signal
            self.sell()#, tp=tp2) 
            
bt = Backtest(df, myStrat, cash=10_000, margin = 1/3, commission=.000)
stat = bt.run()
stat
bt.plot()